# Определение тональности отзывов на банки с помощью векторных представлений слов

Используем логистическую регрессию и векторные представления Navec.



In [ ]:
!pip install pymorphy3
!pip install navec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import pymorphy3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from navec import Navec

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Константы

In [ ]:
random_state = 42
max_review_len = 100
vector_size = 300

## Загружаем и готовим набор данных

In [ ]:
!wget https://www.dropbox.com/s/a9r0b2yj3vqvi13/banks.csv?dl=1 -O banks.csv

--2023-10-21 06:46:15--  https://www.dropbox.com/s/a9r0b2yj3vqvi13/banks.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/a9r0b2yj3vqvi13/banks.csv [following]
--2023-10-21 06:46:16--  https://www.dropbox.com/s/dl/a9r0b2yj3vqvi13/banks.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2d26f1184105d51e48bebdf73a.dl.dropboxusercontent.com/cd/0/get/CGA8CZJ8cJh7etIa71TljS0eKg8A3tbeTk4CtZO_OD4sngIcmb7d5no5BESsmPlLypXsFnr444yKDWmZLFKA3e1qFgJU1hbHyg87XvWOu3GidX-tHQ1kzz3I2V4qCYCpmyGfvd742ukkZLSdoTzvzYoC/file?dl=1# [following]
--2023-10-21 06:46:16--  https://uc2d26f1184105d51e48bebdf73a.dl.dropboxusercontent.com/cd/0/get/CGA8CZJ8cJh7etIa71TljS0eKg8A3tbeTk4CtZO_OD4sngIcmb7d5no5BESsmPlLypXsFnr444yKDWmZLFKA3e1qFgJU1hbHyg87XvWOu3G

In [ ]:
banks = pd.read_csv('banks.csv', sep='\t', index_col='idx');

In [ ]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [ ]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '«', '»', ';', '–', '--']
stop_words = stopwords.words("russian")
morph = pymorphy3.MorphAnalyzer()

In [ ]:
banks['Preprocessed_texts'] = banks.apply(lambda row: preprocess(row['Text'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
banks

,Score,Text,Preprocessed_texts
idx,,,
0,Positive,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,..."
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,..."
2,Positive,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан..."
3,Negative,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре..."
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан..."
...,...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро...","[высокий, надёжность, мкб, порядочность, добро..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаться, офис, чернореченский, 42а, физ..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...,"[попасть, сегодня, очень, неприятный, ситуация..."


## Загружаем предварительно обученные векторы Navec

In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar -O navec_hudlit_v1_12B_500K_300d_100q.tar

--2023-10-21 07:28:33--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  19.8MB/s    in 2.6s    

2023-10-21 07:28:36 (19.8 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [ ]:
navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

## Векторизуем текст

Функция для векторизации текста

In [ ]:
def vectorize_text(txt, navec, max_review_len):
    unk = navec['<unk>']
    text_embeddings = []
    for tocken in txt:
        embedding = navec.get(tocken, unk)
        text_embeddings.append(embedding)
    # Дополняем или обрезаем отзывы для фиксированной длины max_review_len
    l = len(text_embeddings)
    if l > max_review_len:
        text_embeddings = text_embeddings[:max_review_len]
    else:
        text_embeddings.extend([navec['<pad>']] * (max_review_len - l))
    return text_embeddings

Применяем функцию векторизации текста к набору данных отзывов на банки

In [ ]:
banks['Embeddings'] = banks.apply(lambda row: vectorize_text(row['Preprocessed_texts'], navec, max_review_len), axis=1)

In [ ]:
banks

,Score,Text,Preprocessed_texts,Embeddings
idx,,,,
0,Positive,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,...","[[-0.038867943, 0.12470589, 0.45623106, -0.037..."
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,...","[[0.0018760362, 0.47031733, 0.44177133, 0.2619..."
2,Positive,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан...","[[-0.28339294, 0.14809653, -0.08746451, 0.3650..."
3,Negative,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре...","[[-0.2938254, -0.2968786, -0.025342526, 0.0822..."
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан...","[[0.36390617, -0.5678413, -0.24011786, 0.08220..."
...,...,...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро...","[высокий, надёжность, мкб, порядочность, добро...","[[-0.046687543, 0.078019015, -0.3210017, 0.157..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаться, офис, чернореченский, 42а, физ...","[[0.11729202, -0.050626393, 0.29470918, -0.368..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...,"[попасть, сегодня, очень, неприятный, ситуация...","[[-0.34724313, -0.14704202, -0.4983434, -0.222..."


## Готовим данные для обучения

Преобразуем текстовые метки классов в числовые

In [ ]:
mapping = {'Negative': 0, 'Positive': 1}

In [ ]:
banks.replace({'Score': mapping}, inplace=True)

In [ ]:
banks

,Score,Text,Preprocessed_texts,Embeddings
idx,,,,
0,1,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,...","[[-0.038867943, 0.12470589, 0.45623106, -0.037..."
1,0,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,...","[[0.0018760362, 0.47031733, 0.44177133, 0.2619..."
2,1,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан...","[[-0.28339294, 0.14809653, -0.08746451, 0.3650..."
3,0,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре...","[[-0.2938254, -0.2968786, -0.025342526, 0.0822..."
4,0,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан...","[[0.36390617, -0.5678413, -0.24011786, 0.08220..."
...,...,...,...,...
13994,1,"О высокой надёжности МКБ, порядочности и добро...","[высокий, надёжность, мкб, порядочность, добро...","[[-0.046687543, 0.078019015, -0.3210017, 0.157..."
13995,1,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаться, офис, чернореченский, 42а, физ...","[[0.11729202, -0.050626393, 0.29470918, -0.368..."
13996,1,Попала сегодня в очень неприятную ситуацию. Ре...,"[попасть, сегодня, очень, неприятный, ситуация...","[[-0.34724313, -0.14704202, -0.4983434, -0.222..."


### Выделяем данные для обучения и тестирования

In [ ]:
train, test = train_test_split(banks, test_size=0.2)

In [ ]:
train

,Score,Text,Preprocessed_texts,Embeddings
idx,,,,
5202,1,Брала в 2007 году кредит в Хоум кредит банке. ...,"[брать, 2007, год, кредит, хоум, кредит, банк,...","[[-0.2938254, -0.2968786, -0.025342526, 0.3093..."
9032,1,Являюсь клиентом банка уже около 10 лет. Всегд...,"[являться, клиент, банк, около, 10, год, счита...","[[0.7918776, 0.20647636, -0.4752369, -0.021218..."
2974,0,Добрый день сотрудники банка!!! По горячей лин...,"[добрый, день, сотрудник, банк, горячий, линия...","[[-0.06740946, 0.061830815, 0.13073128, 0.1224..."
6033,0,Писал сообщение: http://www.banki.ru/services/...,"[писать, сообщение, http, //www.banki.ru/servi...","[[-0.20394225, -0.5428032, 0.17268088, 0.00080..."
3479,1,Хочу поделиться удачным опытом в получении ипо...,"[хотеть, поделиться, удачный, опыт, получение,...","[[-0.17414474, 0.42308596, 0.39629254, -0.1173..."
...,...,...,...,...
7292,0,21 августа получил sms от Банка Альфа-Банк пре...,"[21, август, получить, sms, банк, альфа-банк, ...","[[0.21431214, 0.37028718, 0.13679631, -0.18653..."
5494,0,Не буду вдаваться в подробности относительно т...,"[быть, вдаваться, подробность, относительно, б...","[[0.2066846, -0.09547346, 0.13113004, -0.19423..."
11313,0,В апреле сего года открыл в банке депозит и по...,"[апрель, сие, год, открыть, банк, депозит, под...","[[0.398813, -0.19200113, -0.21171716, 0.200069..."


In [ ]:
test

,Score,Text,Preprocessed_texts,Embeddings
idx,,,,
5361,0,29.08.15 г. Мною было получено требование от З...,"[29.08.15, г., я, получить, требование, зао, “...","[[0.21431214, 0.37028718, 0.13679631, -0.18653..."
10924,1,Вся процедура ипотечной сделки с использование...,"[весь, процедура, ипотечный, сделка, использов...","[[0.36390617, -0.5678413, -0.24011786, 0.29949..."
8523,1,"Пользуюсь картой плюс, в принципе меня все уст...","[пользоваться, карта, плюс, принцип, устраиват...","[[0.18638138, -0.27080196, -0.3037158, 0.08661..."
12751,1,Хороший банк. Одобрили ипотеку через 5 дней. Д...,"[хороший, банк, одобрить, ипотека, 5, день, до...","[[-0.34992936, -0.3075621, -0.27499628, -0.098..."
12116,0,"Здравствуйте! Не думала, что когда-либо так ра...","[здравствуйте, думать, когда-либо, разочароват...","[[0.21431214, 0.37028718, 0.13679631, 0.126034..."
...,...,...,...,...
6329,0,Продолжаю повествование о рефинансировании кре...,"[продолжать, повествование, рефинансирование, ...","[[0.31074855, -0.4440416, 0.3544376, 0.3093535..."
11684,1,С мужем обратились в Сбербанк с целью получени...,"[муж, обратиться, сбербанк, цель, получение, к...","[[0.05624003, -0.4158787, -0.039217323, -0.021..."
12236,1,"Потерял бумажник, из-за чего пришлось восстана...","[потерять, бумажник, из-за, прийтись, восстана...","[[0.122225314, -0.08969038, -0.06913314, 0.425..."


### Разделяем метки классов и данные для обучения

Данные для обучения

In [ ]:
x_train = np.array(train['Embeddings'].tolist()).reshape(len(train), vector_size * max_review_len)
y_train = train['Score']

In [ ]:
x_train.shape

(11199, 30000)

In [ ]:
y_train

idx
5202     1
9032     1
2974     0
6033     0
3479     1
        ..
7292     0
5494     0
11313    0
6298     1
12420    1
Name: Score, Length: 11199, dtype: int64

Данные для тестирования

In [ ]:
x_test = np.array(test['Embeddings'].tolist()).reshape(len(test), vector_size * max_review_len)
y_test = test['Score']

In [ ]:
x_test.shape

(2800, 30000)

In [ ]:
y_test

idx
5361     0
10924    1
8523     1
12751    1
12116    0
        ..
6329     0
11684    1
12236    1
8469     0
11026    0
Name: Score, Length: 2800, dtype: int64

## Создаем модель машинного обучения

In [ ]:
lr = LogisticRegression(random_state=random_state, max_iter=500)

## Обучаем модель машинного обучения

In [ ]:
lr.fit(x_train, y_train)

LogisticRegression(max_iter=500, random_state=42)

## Оцениваем качество обучения на тестовом наборе данных

Определяем долю правильных ответов (accuracy) на тестовом наборе данных

In [ ]:
lr.score(x_test, y_test)

0.7985714285714286

## Применяем модель для определения тональности отзыва на банк

**Позитивный отзыв**

In [ ]:
positive_text = """Брал кредит в Мегабанке на автомобиль. Выдали за один день. Никаких скрытых комиссий и переплат.
У банка удобное мобильное приложение, через которое можно быстро отправить ежемесячный платеж.
Досрочное гасить начал через три месяца. Я доволен оперативностью и удобством. Огромное спасибо!
"""

Подготовка текста к обработке

In [ ]:
positive_preprocessed_text = preprocess(positive_text, stop_words, punctuation_marks, morph)

In [ ]:
positive_preprocessed_text

['брать',
 'кредит',
 'мегабанк',
 'автомобиль',
 'выдать',
 'день',
 'никакой',
 'скрытый',
 'комиссия',
 'переплата',
 'банк',
 'удобный',
 'мобильный',
 'приложение',
 'который',
 'быстро',
 'отправить',
 'ежемесячный',
 'платёж',
 'досрочный',
 'гасить',
 'начать',
 'месяц',
 'довольный',
 'оперативность',
 'удобство',
 'огромный',
 'спасибо']

In [ ]:
positive_vectorized_text = vectorize_text(positive_preprocessed_text, navec, max_review_len)

In [ ]:
positive_vectorized_text

[array([-2.93825388e-01, -2.96878606e-01, -2.53425259e-02,  3.09353530e-01,
        -7.23195150e-02, -2.71474421e-01,  6.07955158e-01,  1.14302084e-01,
         3.41228068e-01, -1.70095548e-01,  8.59668776e-02, -3.18133205e-01,
         3.51129532e-01,  3.30451339e-01, -2.35911801e-01, -2.38540322e-02,
        -5.06073594e-01, -3.71512145e-01,  3.01847428e-01,  6.99246079e-02,
         6.48074150e-02, -5.98425806e-01,  2.22847804e-01, -1.90987308e-02,
         3.80989730e-01, -2.30993971e-01,  3.91971916e-02, -5.68452403e-02,
        -2.30670050e-01,  3.92794341e-01, -9.85443667e-02,  3.41025591e-01,
         1.18793583e+00,  5.86751163e-01,  4.11680490e-02, -1.36535317e-01,
        -2.75818586e-01, -2.57348180e-01, -3.12133521e-01, -5.10075212e-01,
        -1.25169143e-01, -4.37718391e-01, -2.54204601e-01,  3.08866560e-01,
         4.80890796e-02, -3.17357719e-01, -2.64123619e-01, -9.13212635e-03,
         4.18695211e-01,  2.32695471e-02, -4.17563654e-02, -5.14010429e-01,
        -6.9

In [ ]:
positive_vector = np.array(positive_vectorized_text).reshape(1, vector_size * max_review_len)

In [ ]:
positive_vector.shape

(1, 30000)

Выполняем распознавание

In [ ]:
result = lr.predict(positive_vector)

In [ ]:
result

array([1])

**Негативный отзыв**

In [ ]:
negative_text = """Взял кредит в ТакСебеБанке на автомобиль. В договор включили обязательный контракт
на помощь на дороге, который мне не нужен. Узнал об этом только во время подписания договора, иначе бы отказался.
Альтернативы была страхование жизни, но мне это даже не предложили. Скорее всего, менеджер продвигает
продажи услуг этой компании в ущерб интересов клиента. Как минимум, непорядочно и непрофессионально.
У банка ужасное мобильное приложение, из-за которого с меня взяли штраф 10 тыс.руб. По требованиям
банка после покупки автомобиля в приложении нужно загрузить ПТС. Я загрузил и проверил, что ПТС в приложении есть.
Но через некоторое время ПТС из приложения пропал и с меня взяли штраф. Никому не рекомендую связываться с ТакСебеБанком.
"""

In [ ]:
negative_preprocessed_text = preprocess(negative_text, stop_words, punctuation_marks, morph)
negative_vectorized_text = vectorize_text(negative_preprocessed_text, navec, max_review_len)
negative_vector = np.array(negative_vectorized_text).reshape(1, vector_size * max_review_len)

In [ ]:
result = lr.predict(negative_vector)

In [ ]:
result

array([0])

In [ ]:
result = lr.predict_proba(negative_vector)

In [ ]:
result

array([[0.82079178, 0.17920822]])